In [4]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K

# import the necessary packages
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras.optimizers import SGD
from keras.utils import np_utils
import numpy as np
import cv2

In [5]:
((trainData, trainLabels), (testData, testLabels)) = mnist.load_data()

In [7]:
trainData = trainData.reshape((trainData.shape[0], 28, 28, 1))
testData = testData.reshape((testData.shape[0], 28, 28, 1))
trainData.shape, testData.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [8]:
# scale data to the range of [0, 1]
trainData = trainData.astype("float32") / 255.0
testData = testData.astype("float32") / 255.0

In [10]:
# transform the training and testing labels into vectors in the
# range [0, classes] -- this generates a vector for each label,
# where the index of the label is set to `1` and all other entries
# to `0`; in the case of MNIST, there are 10 class labels
trainLabels = np_utils.to_categorical(trainLabels, 10)
testLabels = np_utils.to_categorical(testLabels, 10)

In [14]:
def build(numChannels, imgRows, imgCols, numClasses, activation="relu", weightsPath=None):
    model = Sequential()
    inputShape = (imgRows, imgCols, numChannels)
    
    # define the first set of CONV => ACTIVATION => POOL layers
    model.add(Conv2D(20, 5, padding="same", input_shape=inputShape))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    # define the second set of CONV => ACTIVATION => POOL layers
    model.add(Conv2D(50, 5, padding="same"))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    # define the first FC => ACTIVATION layers
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation(activation))
    # define the second FC layer
    model.add(Dense(numClasses))
    # lastly, define the soft-max classifier
    model.add(Activation("softmax"))
    
    if weightsPath is not None:
        model.load_weights(weightsPath)
        
    return model

In [13]:
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(lr=0.01)

[INFO] compiling model...


In [15]:
model = build(numChannels=1, imgRows=28, imgCols=28, numClasses=10, weightsPath=None)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [16]:
print("[INFO] training...")
model.fit(trainData, trainLabels, batch_size=128, epochs=20, verbose=1)

[INFO] training...
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
60000/60000 [==============================] - 124s 2ms/step - loss: 1.0513 - acc: 0.7262
Epoch 2/20
60000/60000 [==============================] - 136s 2ms/step - loss: 0.2624 - acc: 0.9217
Epoch 3/20
60000/60000 [==============================] - 136s 2ms/step - loss: 0.1860 - acc: 0.9449
Epoch 4/20
60000/60000 [==============================] - 138s 2ms/step - loss: 0.1455 - acc: 0.9565
Epoch 5/20
60000/60000 [==============================] - 135s 2ms/step - loss: 0.1197 - acc: 0.9645
Epoch 6/20
60000/60000 [==============================] - 155s 3ms/step - loss: 0.1024 - acc: 0.9695
Epoch 7/20
60000/60000 [==============================] - 158s 3ms/step - loss: 0.0898 - acc: 0.9730
Epoch 8/20
60000/60000 [==============================] - 151s 3ms/step - loss: 0.0801 - acc: 0.9758
Epoch 9/20
60000/60000 [==============================] - 157s 3ms/step - loss: 0.0732 - acc: 0.9779
Epoch 10/20
60000/60000 

In [17]:
# show the accuracy on the testing set
print("[INFO] evaluating...")
(loss, accuracy) = model.evaluate(testData, testLabels, batch_size=128, verbose=1)
print("[INFO] accuracy: {:.2f}%".format(accuracy * 100))

[INFO] evaluating...
10000/10000 [==============================] - 9s 899us/step
[INFO] accuracy: 98.46%


In [19]:
print("[INFO] dumping weights to file...")
model.save_weights("lenet_weights.hdf5")

[INFO] dumping weights to file...


In [2]:
# randomly select a few testing digits
for i in np.random.choice(np.arange(0, len(testLabels)), size=(10,)):
    # classify the digit
    probs = model.predict(testData[np.newaxis, i])
    prediction = probs.argmax(axis=1)
    
    image = (testData[i] * 255).astype("uint8")
    image = cv2.merge([image] * 3)
    image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
    
    # show the image and prediction
    cv2.putText(image, str(prediction[0]), (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
    print("[INFO] Predicted: {}, Actual: {}".format(prediction[0], np.argmax(testLabels[i])))
    cv2.imshow("Digit", image)